In [31]:
import os 
import sys 
import json 
import logging 
import builtins 
from typing import List 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

from utils_notebook.utils import ddf, remove_prefix, load_subgraph, remove_keys
from utils_notebook.vega import condition_union, output_chart, apply_css, stack_order_expr, chart, XAXIS_DEFAULTS, possibly_override
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK
from utils_notebook.queries import QueryManager
from utils_notebook.css import css_tooltip_timeseries_multi_colored

import warnings
warnings.filterwarnings('ignore')


In [2]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs) 

In [3]:
@cache
def query_barn(**kwargs): 
    return q.query_barn()

In [4]:
df_barn = query_barn(cache=1)
df_barn = df_barn[['season', 'sprouts', 'sprouts_rinsable']]
df_barn.head()

,season,sprouts,sprouts_rinsable
1,6074,8.643268e+07,0.000000
2,6075,8.644328e+07,0.000000
3,6076,8.654976e+07,2266.788451
4,6077,8.658356e+07,6824.618896
5,6078,8.659188e+07,13760.685470


In [5]:
col_map = {
    'newHarvestablePods': 'pods_harvestable_daily',
    'newHarvestedPods': 'pods_harvested_daily', 
    'podIndex': 'pods_issued_cumulative', 
    'totalHarvestablePods': 'pods_harvestable_cumulative', 
}

In [6]:
@cache
def query_field_daily_snapshots(**kwargs): 
    return q.query_field_daily_snapshots(fields=['season'] + list(col_map.keys())) 

In [7]:
df_field = query_field_daily_snapshots(cache=1).copy()
df_field = df_field.rename(columns=col_map).drop(columns=['timestamp'])
df_field['pods_unharvestable_cumulative'] = df_field.pods_issued_cumulative - df_field.pods_harvestable_cumulative
df_field.tail()

,season,pods_harvestable_daily,pods_harvested_daily,pods_harvestable_cumulative,pods_issued_cumulative,pods_unharvestable_cumulative
336,8049,6349.027387,0.000000,5.759723e+07,8.291974e+08,7.716002e+08
337,8073,8805.577968,0.000000,5.760604e+07,8.292044e+08,7.715983e+08
338,8097,3304.490037,29890.906548,5.760934e+07,8.292106e+08,7.716013e+08
339,8121,12585.135314,0.000000,5.762193e+07,8.292169e+08,7.715950e+08
340,8138,4784.941825,0.000000,5.762671e+07,8.292183e+08,7.715916e+08


In [8]:
@cache
def query_silo_daily_snapshots(**kwargs): 
    return q.query_silo_daily_snapshots() 

In [9]:
# process post-replant silo data (subgraph)
df_silo = query_silo_daily_snapshots()
df_silo = df_silo.rename(columns={"dailyBeanMints": "silo_emissions_daily"})
df_silo['silo_emissions_cumulative'] = df_silo.silo_emissions_daily.cumsum()
df_silo.tail()

,season,silo_emissions_daily,silo_emissions_cumulative
2819,8049,6349.027393,7.629153e+07
2820,8073,8805.577979,7.630033e+07
2821,8097,3304.490041,7.630364e+07
2822,8121,12585.135329,7.631622e+07
2823,8138,4784.941829,7.632101e+07


In [10]:
@cache 
def query_seasons(**kwargs): 
    return q.query_seasons(extra_cols=['beans'])

In [11]:
df_szns = query_seasons(cache=1)

In [12]:
df = df_szns.merge(
    df_barn, how='left', on='season'
).merge(
    df_field, how='left', on='season'
).merge(
    df_silo, how='left', on='season'
)
df.tail()

,season,timestamp,beans,sprouts,sprouts_rinsable,pods_harvestable_daily,pods_harvested_daily,pods_harvestable_cumulative,pods_issued_cumulative,pods_unharvestable_cumulative,silo_emissions_daily,silo_emissions_cumulative
8134,8134,2022-10-31 12:00:11,3.290442e+07,9.385924e+07,2.172736e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8135,8135,2022-10-31 13:00:11,3.290453e+07,9.385924e+07,2.172736e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8136,8136,2022-10-31 14:00:11,3.290464e+07,9.385924e+07,2.172736e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8137,8137,2022-10-31 15:00:11,3.290475e+07,9.385924e+07,2.172736e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8138,8138,2022-10-31 16:00:11,3.290486e+07,9.385924e+07,2.172736e+06,4784.941825,0.0,5.762671e+07,8.292183e+08,7.715916e+08,4784.941829,7.632101e+07


In [13]:
assert len(df) == len(df_szns)
df = df.rename(columns={
    # credit components 
    'sprouts_rinsable': 'fertilized beans', 
    'pods_harvestable_cumulative': 'pods harvestable', 
    'silo_emissions_cumulative': 'silo emissions', 
    # debt components 
    'sprouts': 'unfertilized beans', 
    'pods_unharvestable_cumulative': 'pods unharvestable', 
    # overall 
    'total_debt': 'total debt', 
    'total_credit': 'total credit', 
    'debt_credit_ratio': 'debt credit ratio', 
    'fertilizer_adjusted_pod_rate': 'fertilizer adjusted pod rate', 
    'beans': 'bean_supply',
})
df = df.ffill().fillna(0) # Not technically correct but close enough 
df['total debt'] = (
    df['pods unharvestable'] + df['unfertilized beans']
) 
df['total credit'] = (
    df['fertilized beans'] + df['silo emissions'] + df['pods harvestable']
)
df['debt credit ratio'] = df['total debt'] / df['total credit'] 
df['fertilizer adjusted pod rate'] = df['total debt'] / df['bean_supply'] 
metrics_credit = [
    'silo emissions',
    'pods harvestable',
    'fertilized beans', 
]
metrics_debt = [
    'pods unharvestable', 
    'unfertilized beans', 
]
metrics_credit_debt_aggregate = [
    'total debt', 
    'total credit', 
]
metrics_meta = [
    'debt credit ratio', 
    'fertilizer adjusted pod rate', 
]
metrics = metrics_credit + metrics_debt + metrics_credit_debt_aggregate + metrics_meta
columns = ['timestamp'] + metrics 
df = df[columns]
df = df.resample("W", on="timestamp").last().reset_index()
df = df.dropna()
source = df.melt(
    id_vars=['timestamp'], 
    value_vars=metrics, 
).sort_values(["timestamp", "variable"]).reset_index(drop=True)
print(len(source))
source.head()

459


,timestamp,variable,value
0,2021-08-08,debt credit ratio,0.067543
1,2021-08-08,fertilized beans,0.000000
2,2021-08-08,fertilizer adjusted pod rate,0.021235
3,2021-08-08,pods harvestable,15863.528585
4,2021-08-08,pods unharvestable,2502.841552


In [34]:
# alt.data_transformers.disable_max_rows()
dropdown = alt.binding_select(
    options=['ymd', 'ym'], 
    labels=["weekly", "monthly"], 
    name='aggregation level:'
)
selection = alt.selection_single(
    name="agglevel", 
    fields=['AggLevel'], 
    bind=dropdown, 
    init={"AggLevel": 'ymd'}
)
colors = {
    # credit components 
    'fertilized beans': '#57cc99', # green   
    'pods harvestable': '#38a3a5', # mid blue 
    'silo emissions': '#22577a', # navy blue 
    # debt components 
    'unfertilized beans': "#ef9b20", # Magenta 50
    'pods unharvestable': '#fa4d56', # Red 50
    # overall 
    'total debt': '#9f1853', # Magenta 70 
    'total credit': '#80ed99', # mint green 
    'debt credit ratio': '#ffc300', # gold 
    'fertilizer adjusted pod rate': '#5e60ce' # purple-ish
}
format_decimal = ",d"
format_percent = ".2%"
tooltip_formats = {
    'fertilized beans':  format_decimal,
    'unfertilized beans':  format_decimal,
    'pods harvestable':  format_decimal,
    'silo emissions':  format_decimal,
    'pods unharvestable':  format_decimal,
    'total debt':  format_decimal,
    'total credit':  format_decimal,
    'debt credit ratio': format_percent, 
    'fertilizer adjusted pod rate': format_percent,
}
assert set(colors.keys()) == set(metrics)
assert set(tooltip_formats.keys()) == set(metrics)

def base_hook(c): 
    return (
        c
        .transform_timeunit(
            ymd="yearmonthdate(timestamp)", 
            ym="yearmonth(timestamp)", 
        )
        .transform_calculate(
            tstamp="datum[agglevel.AggLevel]", 
        )
        .transform_aggregate(
            groupby=["tstamp", 'variable'], value='max(value)'
        )
    )



metrics_debt_all = metrics_debt + ['total debt']
metrics_credit_all = metrics_credit + ['total credit'] 
tooltip_metrics = metrics_debt_all + metrics_credit_all + metrics_meta

# top chart layers 
chart_top_debt, selection_nearest = chart(
    source, 
    "tstamp", 
    lmetrics=metrics_debt_all, 
    lstrategy=["stack_bar" for i in range(len(metrics_debt))] + ['line'], 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    title="Beanstalk Credit Profile", 
    colors=colors,
    base_hook=base_hook,
    yaxis_left_kwargs=dict(
        title="BDV", 
        format=".3~s", 
        labelExpr="replace(datum.label, 'G', 'B')"
    ), 
    return_selection=True,
)
chart_top_credit = chart(
    source, 
    "tstamp", 
    lmetrics=metrics_credit_all, 
    lstrategy=["stack_bar" for i in range(len(metrics_credit))] + ['line'], 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    colors=colors, 
    base_hook=base_hook, 
    create_selection=False,
    add_selection=False, 
    selection_nearest=selection_nearest
)
# bottom chart layers 
chart_bot_line = chart(
    source, 
    "tstamp", 
    lmetrics=metrics_meta, 
    lstrategy="line", 
    colors=colors, 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    base_hook=base_hook,
    yaxis_left_kwargs=dict(
        title="Percent", format=",%"
    ), 
    create_selection=False,
    add_selection=True, 
    selection_nearest=selection_nearest
)
chart_bot_point = chart(
    source, 
    "tstamp", 
    lmetrics=metrics_meta, 
    lstrategy="point", 
    lmark_kwargs=dict(point=dict(size=7)), 
    colors=colors, 
    tooltip_metrics=tooltip_metrics, 
    tooltip_formats=tooltip_formats, 
    base_hook=base_hook, 
    hide_legend=True, 
    create_selection=False, 
    add_selection=False,
    selection_nearest=selection_nearest
)
# Full chart view 
c = (
    alt.vconcat(
        alt.layer(chart_top_debt, chart_top_credit),
        alt.layer(chart_bot_line, chart_bot_point),
    )
    .add_selection(selection)
)

css_lines = [
    "div.chart-wrapper { display: flex; flex-direction: column; }", 
    "form.vega-bindings { display: block; order: -1; }", 
    "canvas { order: 1 }", 
    """
    div.vega-bind { 
        display: inline-block; 
        padding: 5px; 
    }
    span.vega-bind-name { 
        font-weight: 500 !important; 
        padding-right: 5px !important; 
    }
    span.vega-bind-name span { 
        font-weight: 600 !important; 
        padding-right: 5px !important; 
    }
    div.vega-bind select { 
        border: .5px solid #000000;
        border-radius: 3px;
    }
    """,
]
css_lines = css_lines + css_tooltip_timeseries_multi_colored(tooltip_metrics, colors) 
css = "\n".join(css_lines)
apply_css("")
# apply_css(css) 
c

alt.VConcatChart(...)

In [ ]:
output_chart(c, css=css) 